In [19]:
import os
access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]
account_id_hedge = os.environ["ACCOUNT_HEDGE"]
import oandapyV20
import oandapyV20.endpoints.transactions as trans
import pandas as pd

In [13]:
client = oandapyV20.API(access_token=access_token)
param = { "id": 900 }
r = trans.TransactionsSinceID(accountID=account_id_demo,  params=param)
trades= client.request(r)

In [33]:
param1 = { "id": 0 }
request_hedge = trans.TransactionsSinceID(accountID=account_id_hedge,  params=param1)
trades_hedge= client.request(request_hedge)
all_transactions_hedge = pd.DataFrame(trades_hedge['transactions'])
all_transactions_hedge ['time'] = pd.to_datetime(all_transactions_hedge ['time'])
all_transactions_hedge 

,id,accountID,userID,batchID,requestID,time,type,divisionID,siteID,accountUserID,...,plHomeConversionCost,baseFinancingHomeConversionCost,guaranteedExecutionFeeHomeConversionCost,homeConversionCost,tradeID,triggerMode,tradesClosed,closedTradeID,tradeCloseTransactionID,distance
0,1,101-004-31059296-003,31059296,1,1682652946359240083,2025-04-04 11:27:44.777790339+00:00,CREATE,4.0,101.0,31059296.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,101-004-31059296-003,31059296,1,1682652946359240083,2025-04-04 11:27:44.777790339+00:00,CLIENT_CONFIGURE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,101-004-31059296-003,31059296,3,1718681743386592802,2025-04-04 11:27:46.532682976+00:00,TRANSFER_FUNDS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,101-004-31059296-003,31059296,4,61357230115111042,2025-04-04 21:22:13.497057574+00:00,LIMIT_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,101-004-31059296-003,31059296,5,79371629377737404,2025-04-04 21:25:13.666076581+00:00,MARKET_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,101-004-31059296-003,31059296,5,79371629377737404,2025-04-04 21:25:13.666076581+00:00,ORDER_CANCEL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,101-004-31059296-003,31059296,7,79372046460821315,2025-04-06 01:02:33.098770020+00:00,MARKET_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,101-004-31059296-003,31059296,7,79372046460821315,2025-04-06 01:02:33.098770020+00:00,ORDER_CANCEL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,101-004-31059296-003,31059296,9,61357653436547922,2025-04-06 01:24:21.678028091+00:00,MARKET_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,101-004-31059296-003,31059296,9,61357653436547922,2025-04-06 01:24:21.678028091+00:00,ORDER_CANCEL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
all_transactions = pd.DataFrame(trades['transactions'])
all_transactions['time'] = pd.to_datetime(all_transactions['time'])
all_transactions

,id,accountID,userID,batchID,requestID,time,type,instrument,units,timeInForce,...,baseFinancingHomeConversionCost,guaranteedExecutionFeeHomeConversionCost,homeConversionCost,tradeID,triggerCondition,triggerMode,distance,tradesClosed,closedTradeID,tradeCloseTransactionID
0,901,101-004-31059296-001,31059296,901,97385412787380355,2025-04-03 04:41:01.361067110+00:00,MARKET_ORDER,EUR_USD,-1000000,FOK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,902,101-004-31059296-001,31059296,901,97385412787380355,2025-04-03 04:41:01.361067110+00:00,ORDER_FILL,EUR_USD,-1000000,NaN,...,0.00000000,0.00000000000000,0.00000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,903,101-004-31059296-001,31059296,901,97385412787380355,2025-04-03 04:41:01.361067110+00:00,TAKE_PROFIT_ORDER,NaN,NaN,GTC,...,NaN,NaN,NaN,902,DEFAULT,NaN,NaN,NaN,NaN,NaN
3,904,101-004-31059296-001,31059296,901,97385412787380355,2025-04-03 04:41:01.361067110+00:00,TRAILING_STOP_LOSS_ORDER,NaN,NaN,GTC,...,NaN,NaN,NaN,902,DEFAULT,TOP_OF_BOOK,0.00055,NaN,NaN,NaN
4,905,101-004-31059296-001,31059296,905,NaN,2025-04-03 05:05:43.489654690+00:00,ORDER_FILL,EUR_USD,1000000,NaN,...,0.00000000000000,0.00000000000000,-0.22595668535940,NaN,NaN,NaN,NaN,"[{'tradeID': '902', 'units': '1000000', 'reali...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,1004,101-004-31059296-001,31059296,1003,61358176051763718,2025-04-07 12:01:02.825442962+00:00,ORDER_FILL,EUR_USD,1000000,NaN,...,0.00000000,0.00000000000000,0.00000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,1005,101-004-31059296-001,31059296,1003,61358176051763718,2025-04-07 12:01:02.825442962+00:00,TAKE_PROFIT_ORDER,NaN,NaN,GTC,...,NaN,NaN,NaN,1004,DEFAULT,NaN,NaN,NaN,NaN,NaN
105,1006,101-004-31059296-001,31059296,1003,61358176051763718,2025-04-07 12:01:02.825442962+00:00,TRAILING_STOP_LOSS_ORDER,NaN,NaN,GTC,...,NaN,NaN,NaN,1004,DEFAULT,TOP_OF_BOOK,0.00050,NaN,NaN,NaN
106,1007,101-004-31059296-001,31059296,1007,NaN,2025-04-07 12:02:00.050398560+00:00,ORDER_FILL,EUR_USD,-1000000,NaN,...,0.00000000000000,0.00000000000000,-1.84725729294720,NaN,NaN,NaN,NaN,"[{'tradeID': '1004', 'units': '-1000000', 'rea...",NaN,NaN


In [5]:
orders = pd.read_csv('/work/order_history.csv')
orders['Time'] = pd.to_datetime(orders['Time'])

In [6]:

all_transactions['id'] = pd.to_numeric(all_transactions['id'])
all_transactions[all_transactions['id'].isin(orders['Order_ID'])].head(2)

,id,accountID,userID,batchID,requestID,time,type,divisionID,siteID,accountUserID,...,rejectReason,gtdTime,guaranteedStopLossOnFill,guaranteedExecutionPremium,longPositionCloseout,positionFinancings,baseHomeConversionCost,tradeReduced,distance,trailingStopLossOnFill
151,152,101-004-31059296-001,31059296,152,115394447449742676,2025-03-19 09:27:01.537154565+00:00,MARKET_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,154,101-004-31059296-001,31059296,154,61351307288090414,2025-03-19 13:07:01.312072734+00:00,MARKET_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
all_transactions[all_transactions['reason'].isin(['STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].sum()

reason
STOP_LOSS_ORDER     -20150.0616
TAKE_PROFIT_ORDER    22577.5745
Name: pl, dtype: float64

In [10]:
all_transactions[['id','type','reason','batchID','orderID','tradeID','pl','time']].head(2)

,id,type,reason,batchID,orderID,tradeID,pl,time
0,1,CREATE,NaN,1,NaN,NaN,NaN,2025-02-12 15:52:59.277325926+00:00
1,2,CLIENT_CONFIGURE,NaN,1,NaN,NaN,NaN,2025-02-12 15:52:59.277325926+00:00


In [11]:
trans_orders = _dntk.execute_sql(
  'WITH trading_orders AS (\n    SELECT \n        CAST(all1.batchID AS INTEGER) as batchID,\n        base.pl,\n        all1.time,\n        date_trunc(\'minute\', all1.time - INTERVAL \'7 minutes\') AS order_time\n    FROM all_transactions base \n    LEFT JOIN all_transactions order_filled \n        ON CAST(base.orderID AS INTEGER) = order_filled.id\n    LEFT JOIN all_transactions all1 \n        ON order_filled.batchID = all1.id\n    WHERE base.reason IN (\'TAKE_PROFIT_ORDER\', \'STOP_LOSS_ORDER\')\n)\nSELECT * FROM trading_orders\nORDER BY batchID ',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
trans_orders

,batchID,pl,time,order_time
0,4,-2.0423,2025-02-12 15:55:23.415344+00:00,2025-02-12 15:48:00+00:00
1,18,-2.7433,2025-02-13 02:15:20.220511+00:00,2025-02-13 02:08:00+00:00
2,22,-0.0079,2025-02-27 01:19:36.388815+00:00,2025-02-27 01:12:00+00:00
3,26,-0.0167,2025-02-27 01:25:41.603451+00:00,2025-02-27 01:18:00+00:00
4,38,-0.0159,2025-02-27 12:47:18.701539+00:00,2025-02-27 12:40:00+00:00
...,...,...,...,...
98,801,0.0006,2025-04-02 22:27:27.901929+00:00,2025-04-02 22:20:00+00:00
99,832,0.0003,2025-04-02 22:42:22.780999+00:00,2025-04-02 22:35:00+00:00
100,841,0.0001,2025-04-02 22:45:55.021637+00:00,2025-04-02 22:38:00+00:00
101,919,1292.8004,2025-04-03 18:56:01.961608+00:00,2025-04-03 18:49:00+00:00


In [12]:
trans_orders

,batchID,pl,time,order_time
0,4,-2.0423,2025-02-12 15:55:23.415344+00:00,2025-02-12 15:48:00+00:00
1,18,-2.7433,2025-02-13 02:15:20.220511+00:00,2025-02-13 02:08:00+00:00
2,22,-0.0079,2025-02-27 01:19:36.388815+00:00,2025-02-27 01:12:00+00:00
3,26,-0.0167,2025-02-27 01:25:41.603451+00:00,2025-02-27 01:18:00+00:00
4,38,-0.0159,2025-02-27 12:47:18.701539+00:00,2025-02-27 12:40:00+00:00
...,...,...,...,...
98,801,0.0006,2025-04-02 22:27:27.901929+00:00,2025-04-02 22:20:00+00:00
99,832,0.0003,2025-04-02 22:42:22.780999+00:00,2025-04-02 22:35:00+00:00
100,841,0.0001,2025-04-02 22:45:55.021637+00:00,2025-04-02 22:38:00+00:00
101,919,1292.8004,2025-04-03 18:56:01.961608+00:00,2025-04-03 18:49:00+00:00


In [13]:
all_transactions['date'] = all_transactions['time'].dt.date
all_transactions = all_transactions[all_transactions['date'] == all_transactions['date'].max()]
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
all_transactions[all_transactions['reason'].isin(['STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].sum()

reason
TAKE_PROFIT_ORDER    2509.6932
Name: pl, dtype: float64

In [15]:
print(all_transactions[all_transactions['reason'].isin(['STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].sum())
print(all_transactions[all_transactions['reason'].isin(['STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].count())

reason
TAKE_PROFIT_ORDER    2509.6932
Name: pl, dtype: float64
reason
TAKE_PROFIT_ORDER    2
Name: pl, dtype: int64


In [17]:
total_pl=all_transactions[all_transactions['reason'].isin(['STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].sum()

In [19]:
total_pl['STOP_LOSS_ORDER'] + total_pl['TAKE_PROFIT_ORDER']

KeyError: 'STOP_LOSS_ORDER'

In [21]:
pd.to_numeric(all_transactions['halfSpreadCost']).sum()

929.3706999999999

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>